In [2]:
# -*- coding: UTF-8 -*-
import time
import pandas as pd
import pickle
import os
from datetime import datetime,timedelta
from sklearn.ensemble import GradientBoostingClassifier
import random
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from operator import itemgetter
import operator
from sklearn.model_selection import KFold
import scipy as sp
import gc
import lightgbm as lgb
from collections import Counter

In [ ]:
# train1129 = pd.read_csv('traindata11-29.csv')
# train1131 = pd.read_csv('testdata11-31.csv')
train1129 = pd.read_csv('trainclean11-29.csv')
train1131 = pd.read_csv('trainclean11-31.csv')
print len(train1129)
print len(train1131)

In [4]:
print train1129.columns

Index([u'Unnamed: 0', u'instand_id', u'label', u'click_time', u'ad_id',
       u'user_id', u'position_id', u'conn_type', u'miui_version', u'ip',
       u'android_version', u'age', u'gender', u'education', u'province',
       u'city', u'device_info', u'app_list', u'advertiser_id', u'compaign_id',
       u'app_id', u'app_description', u'app_category1', u'app_category2',
       u'shopping', u'time'],
      dtype='object')


In [5]:
# t1131 = train1131.drop_duplicates(['label', 'click_time', 'ad_id', 'user_id',
#        'position_id', 'conn_type', 'miui_version', 'ip',
#        'android_version', 'age', 'gender', 'education', 'province',
#        'city', 'device_info', 'app_list', 'advertiser_id', 'compaign_id',
#        'app_id', 'app_description', 'app_category1', 'app_category2',
#        'shopping'])

In [6]:
train_data = pd.read_csv('traindata171.csv')
test_data = pd.read_csv('testdata17.csv')
print len(train_data)
print len(test_data)

1788809
2163130


In [7]:
print train_data.columns

Index([u'instand_id', u'label', u'click_time', u'ad_id', u'user_id',
       u'position_id', u'conn_type', u'miui_version', u'ip',
       u'android_version', u'age', u'gender', u'education', u'province',
       u'city', u'device_info', u'app_list', u'advertiser_id', u'compaign_id',
       u'app_id', u'app_description', u'app_category1', u'app_category2',
       u'shopping', u'time'],
      dtype='object')


In [8]:
#统计app下载量
m = []
for i in train1129['app_list']:
    if (i == '-2'):
        continue
    str1 = i.split(',')
    m += str1

app_dl_1129 = Counter(m)

m2 = []
for i in train1131['app_list']:
    if (i == '-2'):
        continue
    str1 = i.split(',')
    m2 += str1

app_dl_1131 = Counter(m2)

fun1 = lambda x: app_dl_1129[x[0]]
fun2 = lambda x: app_dl_1131[x[0]]

train_data['app_dl'] = train_data[['app_id']].apply(fun1, axis = 1)
test_data['app_dl'] = test_data[['app_id']].apply(fun2, axis = 1)

In [9]:
#统计广告点击量
ad_num1 = Counter(train1129['ad_id'])
ad_num2 = Counter(train1131['ad_id'])

fun1 = lambda x: ad_num1[x[0]]
fun2 = lambda x: ad_num2[x[0]]

train_data['ad_num'] = train_data[['ad_id']].apply(fun1, axis = 1)
test_data['ad_num'] = test_data[['ad_id']].apply(fun2, axis = 1)

In [10]:
#统计app点击量
app_num1 = Counter(train1129['app_id'])
app_num2 = Counter(train1131['app_id'])

fun1 = lambda x: app_num1[x[0]]
fun2 = lambda x: app_num2[x[0]]

train_data['app_num'] = train_data[['app_id']].apply(fun1, axis = 1)
test_data['app_num'] = test_data[['app_id']].apply(fun2, axis = 1)

In [7]:
# #T统计购物次数
# fun2 = lambda x: len(x[0]) if (x[0] != -1) else 0
# test_data['shopping_num'] = test_data[['shopping']].apply(fun2, axis = 1)
# train_data['shopping_num'] = train_data[['shopping']].apply(fun2, axis = 1)

In [11]:
#统计广告转化量
train_label1 = train1129[train1129.label == 1]
test_label1 = train1131[train1131.label == 1]

ad_cnum1 = Counter(train_label1['ad_id'])
ad_cnum2 = Counter(test_label1['ad_id'])

fun1 = lambda x: ad_cnum1[x[0]] 
fun2 = lambda x: ad_cnum2[x[0]] 

train_data['ad_cnum'] = train_data[['ad_id']].apply(fun1, axis = 1)
test_data['ad_cnum'] = test_data[['ad_id']].apply(fun2, axis = 1)

In [12]:
#统计app转化量
# train11291 = train1129[train1129.label == 1]
# test11291 = train1131[train1131.label == 1]

app_cnum1 = Counter(train_label1['app_id'])
app_cnum2 = Counter(test_label1['app_id'])

fun1 = lambda x: app_cnum1[x[0]] 
fun2 = lambda x: app_cnum2[x[0]] 

train_data['app_cnum'] = train_data[['app_id']].apply(fun1, axis = 1)
test_data['app_cnum'] = test_data[['app_id']].apply(fun2, axis = 1)

In [13]:
#统计广告转换率
fun1 = lambda x: ad_cnum1[x[0]]*1.0 / ad_num1[x[0]]*1.0 if (ad_num1[x[0]] != 0) else -1
fun2 = lambda x: ad_cnum2[x[0]]*1.0 / ad_num2[x[0]]*1.0 if (ad_num2[x[0]] != 0) else -1

train_data['ad_crate'] = train_data[['ad_id']].apply(fun1, axis = 1)
test_data['ad_crate'] = test_data[['ad_id']].apply(fun2, axis = 1)

In [14]:
#统计app转换率
fun1 = lambda x: app_cnum1[x[0]]*1.0 / app_num1[x[0]]*1.0 if (app_num1[x[0]] != 0) else -1
fun2 = lambda x: app_cnum2[x[0]]*1.0 / app_num2[x[0]]*1.0 if (app_num2[x[0]] != 0) else -1

train_data['app_crate'] = train_data[['app_id']].apply(fun1, axis = 1)
test_data['app_crate'] = test_data[['app_id']].apply(fun2, axis = 1)

In [15]:
#获取点击的小时
fun3 = lambda x: (((x[0]%10000) - ((x[0]%10000)%100))/100 + (x[0]%10000)%100)*1.0 / 60*1.0
test_data['click_day'] = test_data[['click_time']].apply(fun3, axis = 1)
train_data['click_day'] = train_data[['click_time']].apply(fun3, axis = 1)

In [16]:
#获取广告账户点击量
advertiser_num1 = Counter(train1129['advertiser_id'])
advertiser_num2 = Counter(train1131['advertiser_id'])

fun1 = lambda x: advertiser_num1[x[0]]
fun2 = lambda x: advertiser_num2[x[0]]

train_data['advertiser_num'] = train_data[['advertiser_id']].apply(fun1, axis = 1)
test_data['advertiser_num'] = test_data[['advertiser_id']].apply(fun2, axis = 1)

In [17]:
#获取推广ID点击量
compaign_num1 = Counter(train1129['compaign_id'])
compaign_num2 = Counter(train1131['compaign_id'])

fun1 = lambda x: compaign_num1[x[0]]
fun2 = lambda x: compaign_num2[x[0]]

train_data['compaign_num'] = train_data[['compaign_id']].apply(fun1, axis = 1)
test_data['compaign_num'] = test_data[['compaign_id']].apply(fun2, axis = 1)

In [18]:
#获取广告账户转化量
advertiser_cnum1 = Counter(train_label1['advertiser_id'])
advertiser_cnum2 = Counter(test_label1['advertiser_id'])

fun1 = lambda x: advertiser_cnum1[x[0]] 
fun2 = lambda x: advertiser_cnum2[x[0]] 

train_data['advertiser_cnum'] = train_data[['advertiser_id']].apply(fun1, axis = 1)
test_data['advertiser_cnum'] = test_data[['advertiser_id']].apply(fun2, axis = 1)

In [19]:
#获取推广ID转化量
compaign_cnum1 = Counter(train_label1['compaign_id'])
compaign_cnum2 = Counter(test_label1['compaign_id'])

fun1 = lambda x: compaign_cnum1[x[0]] 
fun2 = lambda x: compaign_cnum2[x[0]] 

train_data['compaign_cnum'] = train_data[['compaign_id']].apply(fun1, axis = 1)
test_data['compaign_cnum'] = test_data[['compaign_id']].apply(fun2, axis = 1)

In [20]:
#获取广告账户转换率
fun1 = lambda x: advertiser_cnum1[x[0]]*1.0 / advertiser_num1[x[0]]*1.0 if (advertiser_num1[x[0]] != 0) else -1
fun2 = lambda x: advertiser_cnum2[x[0]]*1.0 / advertiser_num2[x[0]]*1.0 if (advertiser_num2[x[0]] != 0) else -1

train_data['advertiser_crate'] = train_data[['advertiser_id']].apply(fun1, axis = 1)
test_data['advertiser_crate'] = test_data[['advertiser_id']].apply(fun2, axis = 1)

In [21]:
#获取推广ID转换率
fun1 = lambda x: compaign_cnum1[x[0]]*1.0 / compaign_num1[x[0]]*1.0 if (compaign_num1[x[0]] != 0) else -1
fun2 = lambda x: compaign_cnum2[x[0]]*1.0 / compaign_num2[x[0]]*1.0 if (compaign_num2[x[0]] != 0) else -1

train_data['compaign_crate'] = train_data[['compaign_id']].apply(fun1, axis = 1)
test_data['compaign_crate'] = test_data[['compaign_id']].apply(fun2, axis = 1)

In [22]:
print train_data

         instand_id  label  click_time   ad_id   user_id  position_id  \
0         290000001      0      291116   13744   6265843            2   
1         290000002      0      290651  110136   4508115            2   
2         290000003      0      291157   92285   8550042            2   
3         290000004      0      291359  105747   7863525            2   
4         290000005      0      292101   12410   9174336            2   
5         290000006      0      291705   92285   9430429            2   
6         290000007      0      292117   12410   1326095            2   
7         290000008      0      291252   14499   1817972            2   
8         290000009      1      291929   84510   5337930            2   
9         290000010      0      291702   53251   7474903            2   
10        290000011      0      291225    8524   9248009            2   
11        290000012      0      291006   11368  11074431            2   
12        290000013      0      290308   70192   78

In [23]:
# print train_data.dtypes
# print test_data.columns

train_data.to_csv('train1901.csv', index = False)
test_data.to_csv('test1901.csv', index = False)

In [70]:
#Run gbdt

gdbt = GradientBoostingClassifier()
gdbt = gdbt.fit(train_set, train_label)

gbdt = gdbt.predict_proba(test_set)
gbdt = pd.DataFrame(gbdt)


In [72]:
ins = test_data['instand_id'].copy()
ans = pd.concat([ins, gbdt[1]], axis = 1)
ans.columns = ['instance_id', 'prob']

print ans 

         instance_id      prob
0          320000001  0.129302
1          320000002  0.135952
2          320000003  0.084100
3          320000004  0.146750
4          320000005  0.105627
5          320000006  0.076122
6          320000007  0.139573
7          320000008  0.079745
8          320000009  0.146317
9          320000010  0.080316
10         320000011  0.205742
11         320000013  0.119374
12         320000014  0.133618
13         320000015  0.134151
14         320000016  0.064189
15         320000017  0.094411
16         320000018  0.173301
17         320000019  0.057983
18         320000020  0.136176
19         320000021  0.120763
20         320000022  0.058394
21         320000023  0.161100
22         320000024  0.239734
23         320000025  0.104295
24         320000026  0.074580
25         320000027  0.147243
26         320000028  0.256113
27         320000029  0.167258
28         320000030  0.130040
29         320000031  0.109875
...              ...       ...
2163100 

In [73]:
ans.to_csv('star_0917_01.csv',index = False)